In [2]:
!dvc add midi_data


To track the changes with git, run:

	git add midi_data.dvc

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



In [7]:
!dvc add processed


To track the changes with git, run:

	git add .gitignore processed.dvc

To enable auto staging, run:

	dvc config core.autostage true


⠋ Checking graph



In [4]:
!git add midi_data.dvc .dvcignore

In [5]:
!dvc remote add -d myremote gs://jazz-dvc-bucket

Setting 'myremote' as a default remote.


In [6]:
!dvc push

ERROR: unexpected error - gs is supported, but requires 'dvc-gs' to be installed: No module named 'dvc_gs'

Having any troubles? Hit us up at https://dvc.org/support, we are always happy to help!


In [ ]:
!dvc remote modify myremote version_aware true

In [19]:
!dvc repro

Running stage 'partner_in_rhyme':
> python pipeline/web_scrape_partnersinrhyme.py
Downloaded midi_data/011.mid
Downloaded midi_data/atrain.mid
Downloaded midi_data/again.mid
Downloaded midi_data/count.mid
Downloaded midi_data/anotheru.mid
Downloaded midi_data/allblues.mid
Downloaded midi_data/bluesy2.mid
Downloaded midi_data/birdland.mid
Downloaded midi_data/almonds.mid
Downloaded midi_data/alljarr.mid
Downloaded midi_data/bounce.mid
Downloaded midi_data/dolphnst.mid
Downloaded midi_data/4bros.mid
Downloaded midi_data/crysslnc.mid
Downloaded midi_data/byt.mid
Downloaded midi_data/ambiant.mid
Downloaded midi_data/dontmean.mid
Downloaded midi_data/cherokee.mid
Downloaded midi_data/dgeneric.mid
Downloaded midi_data/bethleem.mid
Downloaded midi_data/birlan2.mid
Downloaded midi_data/chipblue.mid
Downloaded midi_data/bop_on.mid
Downloaded midi_data/ckick.mid
Downloaded midi_data/improv_3.mid
Downloaded midi_data/django.mid
Downloaded midi_data/bigband.mid
Downloaded midi_data/Dfreesax.mid
Do

ERROR: failed to reproduce 'partner_in_rhyme': failed to run: python pipeline/web_scrape_partnersinrhyme.py, exited with 3221225477


In [21]:
import os
from math import floor
floor(len(os.listdir("midi_data"))*0.8)

92

In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from tensorflow.keras.utils import to_categorical
import os
import yaml
import sys
from math import floor


def get_notes(midi_path,split_index_start,split_index_end):
    """ Get all the notes and chords from the midi files """
    notes = []

    for file in os.listdir(midi_path)[split_index_start:floor(len(os.listdir(midi_path))*split_index_end)]:
        file = midi_path + "/" + file
        
        try:
            print("Parsing %s" % file)
            midi = converter.parse(file)

            notes_to_parse = None

            try: # file has instrument parts
                s2 = instrument.partitionByInstrument(midi)
                notes_to_parse = s2.parts[0].recurse() 
            except: # file has notes in a flat structure
                notes_to_parse = midi.flat.notes

            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))

            with open('processed/notes', 'wb') as filepath:
                pickle.dump(notes, filepath) 
           # we want to cache the notes to fetch them during inference

            
        except Exception as e:
            print(file,":",e)
            raise Exception("Exception while parsing")
    
    return notes  


def prepare_sequences(notes, n_vocab, sequence_length):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = sequence_length

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_output = to_categorical(network_output)
    #network_output = network_output / float(n_vocab)

    #network_output = to_categorical(network_output)

    return (network_input, network_output)


def save_files(n_vocab, network_input, network_output):
    # save the input and output files

    if not os.path.exists('processed'):
        os.makedirs('processed')

    numpy.save('processed/network_input.npy', network_input)
    numpy.save('processed/network_output.npy', network_output)

    # save the n_vocab file
    with open('processed/n_vocab', 'wb') as filepath:
        pickle.dump(n_vocab, filepath)

    

    



In [7]:
os.listdir("midi_data")[0:floor(len(os.listdir("midi_data"))*0.01)]

['011.mid',
 '12433-game-system-gameboy-pokemon-trading-card-game-lightning-club-jazz-remix-sequenced-by-dr.mid',
 '12616-game-system-gameboy-turok-2-title-jazz-remix-sequenced-by-black-wing.mid',
 '13283-game-system-nintendo-64-donkey-kong-64-dk-isle-jazz-remix-sequenced-by-dr.mid',
 '13298-game-system-nintendo-64-donkey-kong-64-frantic-factory-jazz-remix-sequenced-by-michael-richardson-michaelrichardsontotalmail.mid',
 '14803-game-system-nintendo-final-fantasy-battle-victory-jazz-remix-v2.mid']

In [8]:
notes = get_notes("midi_data",0,0.01)

Parsing midi_data/011.mid
Parsing midi_data/12433-game-system-gameboy-pokemon-trading-card-game-lightning-club-jazz-remix-sequenced-by-dr.mid
Parsing midi_data/12616-game-system-gameboy-turok-2-title-jazz-remix-sequenced-by-black-wing.mid
Parsing midi_data/13283-game-system-nintendo-64-donkey-kong-64-dk-isle-jazz-remix-sequenced-by-dr.mid
Parsing midi_data/13298-game-system-nintendo-64-donkey-kong-64-frantic-factory-jazz-remix-sequenced-by-michael-richardson-michaelrichardsontotalmail.mid
Parsing midi_data/14803-game-system-nintendo-final-fantasy-battle-victory-jazz-remix-v2.mid


In [9]:
n_vocab = len(set(notes))

In [11]:
n_vocab

51

In [12]:
network_input, network_output = prepare_sequences(notes, n_vocab,100)

In [13]:
network_input.shape

(670, 100, 1)

In [14]:
network_output.shape

(670, 51)

In [2]:
airflow standalone start

Usage: airflow [-h] GROUP_OR_COMMAND ...

Positional Arguments:
  GROUP_OR_COMMAND

    Groups
      config         View configuration
      connections    Manage connections
      dags           Manage DAGs
      db             Database operations
      jobs           Manage jobs
      pools          Manage pools
      providers      Display providers
      roles          Manage roles
      tasks          Manage tasks
      users          Manage users
      variables      Manage variables

    Commands:
      cheat-sheet    Display cheat sheet
      dag-processor  Start a standalone Dag Processor instance
      info           Show information about current Airflow and environment
      kerberos       Start a kerberos ticket renewer
      plugins        Dump information about loaded plugins
      rotate-fernet-key
                     Rotate encrypted connection credentials and variables
      scheduler      Start a scheduler instance
      standalone     Run an all-in-one copy of Airf


airflow command error: unrecognized arguments: start, see help above.


In [79]:
network_input[0]

array([[0.93245779],
       [0.51219512],
       [0.93245779],
       [0.41463415],
       [0.08255159],
       [0.2326454 ],
       [0.71669794],
       [0.67917448],
       [0.42026266],
       [0.53470919],
       [0.4521576 ],
       [0.99624765],
       [0.88930582],
       [0.53470919],
       [0.88930582],
       [0.88930582],
       [0.88930582],
       [0.21575985],
       [0.21575985],
       [0.88930582],
       [0.98686679],
       [0.18386492],
       [0.91932458],
       [0.53658537],
       [0.92682927],
       [0.89868668],
       [0.95497186],
       [0.96247655],
       [0.74296435],
       [0.28705441],
       [0.75046904],
       [0.77110694],
       [0.94371482],
       [0.51594747],
       [0.4521576 ],
       [0.48592871],
       [0.14258912],
       [0.83864916],
       [0.84052533],
       [0.50844278],
       [0.65290807],
       [0.07317073],
       [0.08255159],
       [0.56472795],
       [0.09193246],
       [0.37335835],
       [0.75422139],
       [0.512

In [ ]:

if __name__ == '__main__':

    sequence_length = int(sys.argv[1])
    midi_data_path = sys.argv[2]
    split_index_start = int(sys.argv[3])
    split_index_end = floor(len(os.listdir("midi_data"))*float(sys.argv[4]))
    
    file = open("log", "w")
    sys.stderr.write = file.write

    print("file_name: ", sys.argv[0],
    "\nsequence_length: ", sequence_length,
    "\nmidi_data_path: ", midi_data_path,
    "\nsplit_index_start: ", split_index_start,
    "\nsplit_index_end: ", split_index_end)

    # with open('config.yaml', 'r') as config_file:
    #         config = yaml.safe_load(config_file)

    # sequence_length = config['sequence_length']

    notes = get_notes(midi_data_path,split_index_start,split_index_end)
    # get amount of pitch names
    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab,sequence_length)

    save_files(n_vocab, network_input, network_output) 

In [82]:
remote_url = "https://www.midis101.com/download/4248-jazzy"

In [16]:
from music21 import converter,instrument,note,chord
midi = converter.parse("midi_data/42506-christmas-silent-jazz.mid")

In [17]:

s2 = instrument.partitionByInstrument(midi)
s2.parts[0].recurse()

<music21.stream.iterator.RecursiveIterator for Part:BASS @:0>

In [21]:
for i in midi.flatten().notes:
    print(i)

<music21.note.Note C>
<music21.percussion.PercussionChord [Bass Drum Percussion]>
<music21.chord.Chord B3 D5 E4 G4>
<music21.note.Note G>
<music21.chord.Chord B3 D5 E4 G4>
<music21.note.Note D>
<music21.note.Note C>
<music21.note.Note C>
<music21.note.Note A>
<music21.note.Note E>
<music21.chord.Chord B3 D5 E4 G4>
<music21.note.Note G>
<music21.note.Note A>
<music21.percussion.PercussionChord [Bass Drum Percussion]>
<music21.chord.Chord G4 B4 C4 E5>
<music21.note.Note G>
<music21.chord.Chord A4 C#4 E4>
<music21.note.Note E>
<music21.note.Note A>
<music21.chord.Chord D4 B-3 G#3>
<music21.note.Note B>
<music21.chord.Chord G3 F3>
<music21.note.Note E->
<music21.note.Note C>
<music21.note.Note C>
<music21.percussion.PercussionChord [Bass Drum Percussion]>
<music21.note.Note G>
<music21.chord.Chord A3 C5 E4 G4>
<music21.note.Note G>
<music21.note.Note A>
<music21.note.Note C>
<music21.chord.Chord A3 C5 E4 G4>
<music21.note.Note G>
<music21.note.Note A>
<music21.chord.Chord G3 B4 C4 E4>
<mus

In [18]:
for i in s2.parts:
    print(i)

<music21.stream.Part BASS>
<music21.stream.Part Acoustic Bass>
<music21.stream.Part>
<music21.stream.Part PIANO>
<music21.stream.Part Piano>
<music21.stream.Part MUT. TRUMPET>
<music21.stream.Part Trumpet>


In [83]:
import requests
data = requests.get(remote_url)
# Save file data to local copy
with open("random.mid", 'wb')as file:
    file.write(data.content)

In [90]:
len(os.listdir("midi_data"))

603

In [87]:
import os
import requests
import pickle

def download_midi(url, session):
    url = "https://www.midis101.com" + url
    # Extract the filename from the URL
    filename = url.split("/")[-1] + ".mid"
    print(f"Downloading {url}")
    # Check if the file already exists in the directory
    if os.path.isfile(f"midi_data/{filename}"):
        print(f"Skipping {filename} (already exists)")
        return

    with session.get(url) as response:
        # Save the MIDI file to the "data" folder
        with open(f"midi_data/{filename}", "wb") as file:
            file.write(response.content)

        print(f"Downloaded {filename}")

def load_links_from_pickle():
    with open("pipeline/download_links.pkl", "rb") as file:
        download_links = pickle.load(file)
    return download_links

def main():
    # Load download links from the pickle file
    download_links = load_links_from_pickle()

    # Create the "data" folder if it doesn't exist
    if not os.path.exists("midi_data"):
        os.makedirs("midi_data")

    # Download MIDI files from the URLs in download_links
    with requests.Session() as session:
        for url in download_links:
            download_midi(url, session)

    print("MIDI files downloaded successfully.")

if __name__ == "__main__":
    main()


Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
Writing 94440-jazzslow.mid
W

KeyboardInterrupt: 